In [2]:
import pandas as pd
extracted = pd.read_csv("./files/extracted_text.csv")
extracted.head(1)

,imageid,text,recipient,recipientaddress,invoicenos,invoicedate,duedate,Balance
0,2,Invoice\n\nStanford Plumbing & Heating\nbeotts...,Allen Smith,"123 Madison drive Seattle, WA",INVO2081,"Jun 14,2018","Jun 19,2018","2,688.00"


In [3]:
print(len(extracted))

200


In [4]:
import spacy

nlp = spacy.load("en_core_web_sm")
corpus = []
i=0

while i < (len(extracted)):
    text = extracted['text'][i]
    doc = nlp(text)
    for sent in doc.sents:
        corpus.append([i+1,sent.text])
    i=i+1

print("finished")
    


finished


In [5]:
print(corpus)

[[1, 'Invoice\n\nStanford Plumbing & Heating\nbeottsanforepLamarg cm\n12s sadiean dre\nte wh\n20\nPhone sib.'], [1, 'ches\n\nNumber mncuoe\n\nbate junta, 2018\nbee unt, 2018\n\nAllen Smith\n\nsma crm\n\nBe pwvat\nte, ws\n\name: yf 1920\n\nlestlled nave kitchen sine nce suc\nacc me about ne "9 aed mn ety plow ana mane 54 ¢~ ens cour tetop\nroca snk 28 feovee 1 sue\npearing\nNese sma thesmnoctat fesoce 1 fasuen\nne "222 088,\n‘worcester Greenscar 3th sramuee 1 stones\nBrera)\nsubtotal seanuce\ncsinss sesuce\nona seasuce\nBalance Due $2,688.00\n\nNotes\nease pay vat 20 day by Fay: est@scanfoepLamoig com\n\nInark yea for you busiest\n'], [2, 'Invoice\n\nStanford Plumbing & Heating\nbeottsanforepLamarg cm\n12s sadiean dre\nte wh\n20\nPhone sib.'], [2, 'ches\n\nNumber mncuoe\n\nbate junta, 2018\nbee unt, 2018\n\nAllen Smith\n\nsma crm\n\nBe pwvat\nte, ws\n\name: yf 1920\n\nlestlled nave kitchen sine nce suc\nacc me about ne "9 aed mn ety plow ana mane 54 ¢~ ens cour tetop\nroca snk 28 feovee

In [6]:
corpusdf = pd.DataFrame(corpus,columns=["id","sentence"])

In [7]:
corpusdf.dtypes


id           int64
sentence    object
dtype: object

In [8]:
corpusdf.head(1)

,id,sentence
0,1,Invoice\n\nStanford Plumbing & Heating\nbeotts...


In [15]:
nlp = spacy.blank("en")

ruler = nlp.add_pipe("entity_ruler")

patterns = [
                {"label": "ORG", "pattern": "Stanford Plumbing & Heating"},
                {"label": "recipient", "pattern": "Allen Smith"},
                {"label": "recipientaddress", "pattern": "123 Madison drive Seattle, WA"},
                {"label": "invoicenos", "pattern": "INVO2081"},
                {"label": "invoicenos", "pattern": "#INVO2081"},
                {"label": "invoicedate", "pattern": "11/11/18"},
                {"label": "invoicedate", "pattern": "Jun 14,2018"},
                {"label": "duedate", "pattern": "Jun 19,2018"},
                {"label": "duedate", "pattern": "12/01/28"},
                {"label":"Balance","pattern":"$2,688.00"}
            ]

ruler.add_patterns(patterns)

i = 0
entities = []
TRAIN_DATA = []

for row in corpusdf:
    sentence = corpusdf['sentence'][i]
    doc = nlp(sentence)
    for ent in doc.ents:
        entities.append([ent.label_,ent.text, ent.start_char, ent.end_char])
    TRAIN_DATA.append([sentence, {"entities": entities}]) 
    i = i+1 

# ent.text,ent.label_
# print (TRAIN_DATA)
print('finished')


finished


In [12]:
enlab = pd.DataFrame(entities,columns=['label','text','start','end']) 
Trained = pd.DataFrame(TRAIN_DATA,columns=['sentence','entities'])
Trained.to_csv("./files/trained.csv", index=False)
Trained.to_json("./files/trained1.json", index=False)
enlab.to_json('./files/entitylabel.json',index=False)


In [8]:
Trained.head(1)

,sentence,entities
0,Invoice\n\nStanford Plumbing & Heating\nbeotts...,"{'entities': [[9, 36, 'ORG'], [53, 64, 'recipi..."


In [10]:
import srsly
import typer
import warnings
from pathlib import Path

import spacy
from spacy.tokens import DocBin

def convert(lang: str, TRAIN_DATA, output_path: Path):
    nlp = spacy.blank(lang)
    db = DocBin()
    for text, annot in TRAIN_DATA:
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label)
            if span is None:
                msg = f"Skipping entity [{start}, {end}, {label}] in the following text because the character span '{doc.text[start:end]}' does not align with token boundaries:\n\n{repr(text)}\n"
                warnings.warn(msg)
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    db.to_disk(output_path)

In [11]:
convert("en", TRAIN_DATA, "./data/train.spacy")
convert("en", TRAIN_DATA, "./data/valid.spacy")

/tmp/ipykernel_57832/2769882645.py:19: UserWarning: Skipping entity [53, 64, recipient] in the following text because the character span 'arg cm
12s ' does not align with token boundaries:

'Invoice\n\nStanford Plumbing & Heating\nbeottsanforepLamarg cm\n12s sadiean dre\nte wh\n20\nPhone sib.'

  warnings.warn(msg)
/tmp/ipykernel_57832/2769882645.py:19: UserWarning: Skipping entity [393, 402, Balance] in the following text because the character span '' does not align with token boundaries:

'Invoice\n\nStanford Plumbing & Heating\nbeottsanforepLamarg cm\n12s sadiean dre\nte wh\n20\nPhone sib.'

  warnings.warn(msg)
/tmp/ipykernel_57832/2769882645.py:19: UserWarning: Skipping entity [9, 36, ORG] in the following text because the character span 'ber mncuoe

bate junta, 201' does not align with token boundaries:

'ches\n\nNumber mncuoe\n\nbate junta, 2018\nbee unt, 2018\n\nAllen Smith\n\nsma crm\n\nBe pwvat\nte, ws\n\name: yf 1920\n\nlestlled nave kitchen sine nce suc\nacc me about ne "9 

In [12]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [14]:
!python -m spacy train config.cfg --output models/output

ℹ Saving to output directory: models/output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     55.14    0.00    0.00    0.00    0.00
103     200       1272.82   1310.86  100.00  100.00  100.00    1.00
303     400          0.00      0.00  100.00  100.00  100.00    1.00
503     600          0.00      0.00  100.00  100.00  100.00    1.00
703     800          0.00      0.00  100.00  100.00  100.00    1.00
903    1000          0.00      0.00  100.00  100.00  100.00    1.00
1103    1200          0.00      0.00  100.00  100.00  100.00    1.00
1303    1400          0.00      0.00  100.00  100.00  100.00    1.00
1503    1600          

In [ ]:
# In case it fails
# !python -m spacy debug data config.cfg
